In [1]:
import pandas as pd
from json import load
from config.config import lee_archivo_csv, configuracion, diccionario_precios_m2
import src.limpieza_columnas
import src.calculo_columnas
from inspect import isfunction

In [2]:
def columnas_limpias(df):
    try:
        lista_variables = configuracion['variables_en_df']
        for var in lista_variables:
            limpiador = getattr(src.limpieza_columnas, var)
            df[var] = df[var].apply(limpiador)    
        return df[lista_variables]
    except Exception as ex :
        plantilla = "Ocurrió una excepción de tipo {0}. Argumentos:\n{1!r}"
        mensaje = plantilla.format(type(ex).__name__, ex.args)
        print(mensaje)
        print("Error en 'columnas_limpias'")

In [3]:
def calcula_variables(lista_var,df):
    for var in lista_var:
        calculador = getattr(src.calculo_columnas, var)
        df = calculador(df)
    return df

In [4]:
def agrupar_estado(df):
    def actualiza_diccionario(diccionario):
        return {clave: (pd.Series.mode  if cadena == "mode" else cadena) for clave, cadena in diccionario.items()}
    diccionario = actualiza_diccionario(configuracion['agrupar'])
    return df.groupby('uv', as_index = True).agg(diccionario)

In [5]:
df = lee_archivo_csv('./data/datos_pisos.csv')
df = df.set_index('uv')
columnas = list(df.columns)

df_limpio = columnas_limpias(df)
df_calculado = calcula_variables(configuracion['variables_a_calcular'],df_limpio)
df_estado_unico = agrupar_estado(df_calculado)

/mnt/c/Users/emili/Ironhack/mis_proyectos/clikalia/src/calculo_columnas.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NS'] = df.orientacion.apply(es_NS)
/mnt/c/Users/emili/Ironhack/mis_proyectos/clikalia/src/calculo_columnas.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['EO'] = df.orientacion.apply(es_EO)
/mnt/c/Users/emili/Ironhack/mis_proyectos/clikalia/src/calculo_columnas.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [6]:
df_estado_unico

,estado_x,habitaciones,banos,metros,fecha_construccion,NS,EO,situacion,portal,ascensor,...,piscina,garaje,tejado,calefaccion,jardin,aire_acondicionado,tipo_vivienda,precio,gastos_comunitarios,planta
uv,,,,,,,,,,,,,,,,,,,,,
UV/2020/000003,1.0,2.0,1.0,45.0,1960.0,N,,3.0,0,1,...,0,0,0,0,0,0,PISO,160.0,0.00,9.0
UV/2020/000004,1.0,2.0,1.0,67.0,1966.0,,O,1.0,0,1,...,0,0,0,0,0,0,PISO,125.0,0.00,1.0
UV/2020/000006,1.0,2.0,1.0,62.0,1960.0,S,,3.0,0,0,...,0,0,0,0,0,0,PISO,NaN,0.00,2.0
UV/2020/000007,1.0,3.0,1.0,78.0,1983.0,,E,3.0,0,1,...,0,0,0,0,0,0,PISO,159.0,0.00,9.0
UV/2020/000008,1.0,3.0,1.0,84.0,1980.0,,O,3.0,0,1,...,0,0,0,0,0,0,PISO,170.0,0.00,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UV/2021/000444,1.0,3.0,1.0,62.0,1966.0,S,E,3.0,1,0,...,0,0,0,0,0,0,PISO,120.0,0.00,5.0
UV/2021/000448,1.0,2.0,1.0,56.0,1970.0,S,O,1.0,0,0,...,0,0,0,0,0,0,PISO,130.0,30.47,3.0
UV/2021/000451,1.0,2.0,1.0,47.0,1960.0,N,O,1.0,1,0,...,0,0,0,0,0,0,PISO,100.0,35.00,1.0
